# Entrega 1
#### ManyBody - Claudia Garcia, Adrià Rojo - Abril 2025

In [2]:
import numpy as np
import sympy as sp
import scipy.interpolate as scint

scattering_length, width, step, atom_numbers, time_step, alpha, iterations = \
    0.00433, 700, 0.015, 1000000, 0.00005, 0.3, 70000
    # 0.00433, 400, 0.015, 10000, 0.0001, 0.8, 40000
    # 0.00433, 600, 0.015, 100000, 0.0001, 0.4, 50000
    # 0.00433, 400, 0.020, 1000, 0.0001, 0.5, 50000
    # 0.00433, 300, 0.020, 100, 0.0001, 0.5, 50000  

alpha2 = alpha*alpha
cvar = 2 * np.power(alpha, 3/2)/ np.power(np.pi, 1/4)
interaction = 0
density_param = atom_numbers*scattering_length*scattering_length*scattering_length

rs = [i * step for i in range(width)]
r2s = [r*r for r in rs]
rs_r2s = zip(rs, r2s)
psi = [cvar*r*np.exp(-0.5*alpha2*r2) for (r, r2) in rs_r2s]


In [ ]:
psi = []